### Create gold tables
1. fact_ratings (tconst, average_ratings, num_votes)
2. dim_title(tconst, title, year, genre, runtime)
3. bridge_title_person(tconst, nconst, role)
4. dim_person (nconst, name, professions)

In [0]:
USE CATALOG imdb_dev;

In [0]:
SELECT current_catalog();

##### Create table fact_ratings (tconst, average_ratings, num_votes)


In [0]:
CREATE OR REPLACE TABLE imdb_dev.gold.tb_movie_ratings AS
SELECT
  tconst,
  COALESCE(average_rating, 0.0) AS average_rating,
  COALESCE(num_votes, 0) AS num_votes
FROM imdb_dev.silver.title_ratings;

##### Create table dim_title(tconst, title, year, genre, runtime)


In [0]:
CREATE OR REPLACE TABLE imdb_dev.gold.tb_movie_titles AS
SELECT 
       mv.tconst,
       mv.primary_title AS title,
       mv.start_year AS year,
       mv.genres,
       mv.runtime_minutes,
       tr.average_rating,
       tr.num_votes
FROM imdb_dev.silver.movies mv
LEFT JOIN imdb_dev.silver.title_ratings tr
ON mv.tconst = tr.tconst;

##### Create table bridge_title_person(tconst, nconst, role)

In [0]:
-- Use only relavant roles ('director','writer','actor','actress','producer')
CREATE OR REPLACE TABLE imdb_dev.gold.tb_title_person AS
SELECT DISTINCT
  p.tconst,
  p.nconst,
  LOWER(p.category) AS role
FROM imdb_dev.silver.title_principals p
WHERE p.category IN ('director','writer','actor','actress','producer');

##### Create table dim_person (nconst, name, professions)

In [0]:
-- Create person table using roles selected (for optimization)
CREATE OR REPLACE TABLE imdb_dev.gold.tb_person AS
WITH people_in_scope AS (
  SELECT DISTINCT nconst
  FROM imdb_dev.gold.bridge_title_person
)
SELECT
  n.nconst,
  n.primary_name AS name,
  n.birth_year,
  n.death_year,
  primary_profession AS professions
FROM imdb_dev.silver.name_basics n
JOIN people_in_scope s USING (nconst);